In [ ]:
import socket
import numpy as np
from math import dist
from python_tsp.heuristics import solve_tsp_simulated_annealing
import math

PORT = 12345  # Define the port number the server will use for communication.

# Function to generate random node coordinates within specified limits
def generate_nodes(coordinate, computers, nodes):
    full_coordinates = np.empty((0, 2), float)  # Initialize an empty array to store the full set of coordinates (x, y).

    # Create sections in the x and y dimensions based on the coordinate limit.
    x_section_arr = np.linspace(0, coordinate, 3)  # Create 3 equally spaced sections along the x-axis.
    y_section_arr = np.linspace(0, coordinate, num=int(3 + math.ceil((computers - 4) / 2)))  # Create a variable number of sections along the y-axis.

    if computers % 2 == 0:  # Check if the number of computers is even.
        # Loop through each x and y section to generate nodes.
        for x_section in range(len(x_section_arr) - 1):
            for y_section in range(len(y_section_arr) - 1):
                x_nodes_coordinates = np.random.uniform(  # Generate random x-coordinates within the current x section.
                    x_section_arr[x_section],
                    x_section_arr[x_section + 1],
                    size=(int(nodes / computers), 1)  # Generate a number of coordinates equal to nodes divided by computers.
                )
                y_nodes_coordinates = np.random.uniform(  # Generate random y-coordinates within the current y section.
                    y_section_arr[y_section],
                    y_section_arr[y_section + 1],
                    size=(int(nodes / computers), 1)
                )
                col = np.column_stack((x_nodes_coordinates, y_nodes_coordinates))  # Combine the x and y coordinates into a single array.
                full_coordinates = np.append(full_coordinates, col, axis=0)  # Append the new coordinates to the full set.

    else:  # If the number of computers is odd.
        # Similar logic as the even case, but allows one extra section for y-coordinates.
        for x_section in range(len(x_section_arr) - 1):
            for y_section in range(len(y_section_arr) - 2):
                x_nodes_coordinates = np.random.uniform(  # Generate random x-coordinates as before.
                    x_section_arr[x_section],
                    x_section_arr[x_section + 1],
                    size=(int(nodes / computers), 1)
                )
                y_nodes_coordinates = np.random.uniform(  # Generate random y-coordinates as before.
                    y_section_arr[y_section],
                    y_section_arr[y_section + 1],
                    size=(int(nodes / computers), 1)
                )
                col = np.column_stack((x_nodes_coordinates, y_nodes_coordinates))  # Combine coordinates.
                full_coordinates = np.append(full_coordinates, col, axis=0)  # Append to full coordinates.

        # Generate one more set of nodes for the last y section not covered in the previous loop.
        x_nodes_coordinates = np.random.uniform(0, coordinate, size=(int(nodes / computers), 1))  # Random x-coordinates in the full range.
        y_nodes_coordinates = np.random.uniform(y_section_arr[-2], y_section_arr[-1], size=(int(nodes / computers), 1))  # Random y-coordinates in the last section.
        col = np.column_stack((x_nodes_coordinates, y_nodes_coordinates))  # Combine these coordinates.
        full_coordinates = np.append(full_coordinates, col, axis=0)  # Append to the full set.

    return full_coordinates  # Return the complete set of generated node coordinates.

# Function to calculate the distance matrix between nodes
def calcute_distance(arr):  # (Note: "calcute" seems to be a typo, should be "calculate")
    distances = np.empty((0, len(arr)), float)  # Initialize an empty array to hold distances.
    for row in range(len(arr)):  # Loop through each node to calculate its distances to all other nodes.
        temp_dist = np.empty((0, len(arr)), float)  # Prepare an array to store distances from the current node.
        for col in range(len(arr)):  # Loop through each node again.
            point_1 = arr[row][0], arr[row][1]  # Get the coordinates of the current node.
            point_2 = arr[col][0], arr[col][1]  # Get the coordinates of the node we are comparing against.
            temp_dist = np.insert(temp_dist, col, dist(point_1, point_2))  # Calculate and store the distance between the two points.
        distances = np.r_[distances, [temp_dist]]  # Append this row of distances to the distances array.

    return distances  # Return the complete distance matrix.

# Main server function
def start_server(coordinate, computers, nodes):  # Function to start the server and handle connections.
    all_coordinates = generate_nodes(coordinate, computers, nodes)  # Generate random node coordinates.
    full_coordinates = np.array_split(all_coordinates, computers)  # Split the coordinates array into sub-arrays for each computer.

    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_socket:  # Create a TCP socket for the server.
        server_socket.bind(('localhost', PORT))  # Bind the socket to the localhost on the specified port.
        server_socket.listen()  # Set the server to listen for incoming connections.
        print("Server listening on port", PORT)  # Notify that the server is running.

        while True:  # Infinite loop to accept clients.
            conn, addr = server_socket.accept()  # Wait for a client to connect.
            with conn:  # Ensure the connection is properly closed after use.
                print('Connected by', addr)  # Output the address of the connected client.
                while True:  # Loop to handle client requests.
                    data = conn.recv(1024)  # Receive data sent by the client.
                    if not data:  # If no data is received, break the loop (client disconnected).
                        break

                    client_id = int(data.decode('utf-8'))  # Decode the received bytes to get the client ID.
                    temp_arr = full_coordinates[client_id]  # Get the corresponding coordinates for this client ID.
                    distance_matrix = calcute_distance(temp_arr)  # Calculate the distance matrix for the subset of coordinates.
                    permutation, _ = solve_tsp_simulated_annealing(distance_matrix)  # Solve the TSP using simulated annealing.
                    path = temp_arr[permutation]  # Get the ordered path based on the permutation.

                    # Send the path back to the client as bytes.
                    conn.sendall(path.astype(np.float32).tobytes())  # Convert the path to bytes and send back to the client.

# Entry point for the server
if __name__ == "__main__":  # If this script is executed directly (not imported).
    coordinate = int(input("Enter the coordinate limit (e.g., 200): "))  # Prompt for coordinate limit.
    computers = int(input("Enter the number of computers (e.g., 10): "))  # Prompt for the number of computers.
    nodes = int(input("Enter the number of nodes (e.g., 500): "))  # Prompt for the number of nodes to generate.
    start_server(coordinate, computers, nodes)  # Start the server with the provided parameters.

Enter the coordinate limit (e.g., 200):  200
Enter the number of computers (e.g., 10):  8
Enter the number of nodes (e.g., 500):  500


Server listening on port 12345
Connected by ('127.0.0.1', 62340)
Connected by ('127.0.0.1', 62342)
Connected by ('127.0.0.1', 62343)
Connected by ('127.0.0.1', 62344)
Connected by ('127.0.0.1', 62345)
Connected by ('127.0.0.1', 62346)
Connected by ('127.0.0.1', 62347)
Connected by ('127.0.0.1', 62348)
Connected by ('127.0.0.1', 62349)
